# Install

In [ ]:
import os
import shutil

In [ ]:
# check if `pytorch-CycleGAN-and-pix2pix` is already cloned
if not os.path.exists('pytorch-CycleGAN-and-pix2pix'):
	!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix

In [ ]:
os.chdir('./pytorch-CycleGAN-and-pix2pix')

In [ ]:
!pwd

# Datasets

Put the dataset in the `pytorch-CycleGAN-and-pix2pix/datasets` folder

(We have finished this part at the previous step when running `preprocess_dataset.ipynb`)

Each dataset should have the following directory structure:

```
datasets
└── testing_dataset
	├── testA
```

# Testing (Baseline)
convert the testA images to domainB images

use the trained model to inference the testA images

## Load testing data folder

In [ ]:
test_dir = './datasets/public_testing_dataset_1'
# the directory may only contain `testA` subfolder

## Check the weights
After training, you should have a folder with the weights of the model. 

It should be located in the `pytorch-CycleGAN-and-pix2pix/checkpoints` folder.

For example, in our previous training, we have the following weights:
`checkpoints/ROAD_RIVER_pix2pix/latest_net_G.pth`

In [ ]:
!ls checkpoints/

In [ ]:
# if there is no pre-trained model, use our pre-trained model
if not os.path.exists('./checkpoints/ROAD_RIVER_pix2pix'):
    !bash ../scripts/download_pretrained_weight.sh

## Inference with the pre-trained model with single mode

- `--dataroot`: the folder where the testing data is located
- `--name`: the name of the model
- `--model`: the model mode
- `--netG`: the weights of the generator
- `--direction`: the direction of the model
- `--dataset_mode`: single (which we don't need to prepare the paired data)
- `--num_test`: the number of testing data (default is 50)

In [ ]:
load_size = 256
! python test.py --dataroot ./datasets/public_testing_dataset_1/testA/ --name ROAD_RIVER_pix2pix --model test --netG unet_256 --direction AtoB --dataset_mode single --load_size {load_size} --norm batch --num_test 1000 

## Transform the results into AI CUP format

The results are stored in `./results/ROAD_RIVER_pix2pix/test_latest/images/`.

And there are 2 types of results:
- `{Prefix}_real` (domainA)
- `{Prefix}_fake` (domainB)

Store the `{Prefix}_fake.png` as `{Prefix}.jpg` to `./ROAD_RIVER_pix2pix/test_latest/submission/`.

In [ ]:
# store the fake images to the `./results/ROAD_RIVER_pix2pix/test_latest/submission/` folder
import os
import cv2
import shutil

source_folder = './results/ROAD_RIVER_pix2pix/test_latest/images'
target_folder = './results/ROAD_RIVER_pix2pix/test_latest/submission'

if not os.path.exists(target_folder):
    os.makedirs(target_folder)

for image_name in os.listdir(source_folder):
    if 'fake' in image_name:
        new_name = image_name.replace('_fake.png', '.jpg')
        shutil.copy(os.path.join(source_folder, image_name), os.path.join(target_folder, new_name))

In [ ]:
# resize the image as 420x240
import os
import cv2

for image_name in os.listdir(target_folder):
    img = cv2.imread(os.path.join(target_folder, image_name))
    img = cv2.resize(img, (420, 240))
    cv2.imwrite(os.path.join(target_folder, image_name), img)
print("Finished resizing images")
print(f"Size: {len(os.listdir(target_folder))}")

In [ ]:
# zip the fake images
shutil.make_archive(target_folder, 'zip', target_folder)

# Test with Enhanced Method (2 domain-specific models)
convert the `test_RIVER/testA` and `test_ROAD/testA` images to domainB images

use the trained model to inference the testA images

## Test The Model

In [ ]:
# change directory to the pix2pix code
import os

try:
	os.chdir('./pytorch-CycleGAN-and-pix2pix')
except:
    print("Already in the correct directory")

## Load testing data folder

In [ ]:
test_road_dir = './datasets/test_ROAD/testA'
test_river_dir = './datasets/test_RIVER/testA'

road_model = 'ROAD_pix2pix'
river_model = 'RIVER_pix2pix'

## Inference with domain-specific models in single mode

In [ ]:
# test the 2 dataset in single mode
! python test.py --dataroot {test_road_dir} --name {road_model} --model test --netG unet_256 --direction AtoB --dataset_mode single --norm batch --num_test 1000
! python test.py --dataroot {test_river_dir} --name {river_model} --model test --netG unet_256 --direction AtoB --dataset_mode single --norm batch --num_test 1000

## Transform the results into AI CUP format

The results are stored in 
- `./results/ROAD_pix2pix/test_latest/images/`
- `./results/RIVER_pix2pix/test_latest/images/`

And there are 2 types of results in each folder:
- `{Prefix}_real` (domainA)
- `{Prefix}_fake` (domainB)

Store the `{Prefix}_fake.png` as `{Prefix}.jpg` to `./domain_type/test_latest/submission/`.

That is to say
- `./ROAD_pix2pix/test_latest/images/{Prefix}_fake.png` -> `./ROAD_pix2pix/test_latest/submission/{Prefix}.jpg`

- `./RIVER_pix2pix/test_latest/images/{Prefix}_fake.png` -> `./RIVER_pix2pix/test_latest/submission/{Prefix}.jpg`

In [ ]:
# ROAD_pix2pix
# store the fake images to the `./results/ROAD_pix2pix/test_latest/submission/` folder
import os
import shutil

source_folder = './results/ROAD_pix2pix/test_latest/images'
target_folder = './results/ROAD_pix2pix/test_latest/submission'

if not os.path.exists(target_folder):
    os.makedirs(target_folder)

for image_name in os.listdir(source_folder):
    if 'fake' in image_name:
        new_name = image_name.replace('_fake.png', '.jpg')
        shutil.copy(os.path.join(source_folder, image_name), os.path.join(target_folder, new_name))

In [ ]:
# resize the image as 420x240
import os
import cv2

for image_name in os.listdir(target_folder):
    img = cv2.imread(os.path.join(target_folder, image_name))
    img = cv2.resize(img, (420, 240))
    cv2.imwrite(os.path.join(target_folder, image_name), img)
print("Finished resizing images")
print(f"Size: {len(os.listdir(target_folder))}")

In [ ]:
# RIVER_pix2pix
# store the fake images to the `./results/ROAD_pix2pix/test_latest/submission/` folder
import os
import shutil

source_folder = './results/RIVER_pix2pix/test_latest/images'
target_folder = './results/RIVER_pix2pix/test_latest/submission'

if not os.path.exists(target_folder):
    os.makedirs(target_folder)

for image_name in os.listdir(source_folder):
    if 'fake' in image_name:
        new_name = image_name.replace('_fake.png', '.jpg')
        shutil.copy(os.path.join(source_folder, image_name), os.path.join(target_folder, new_name))

In [ ]:
# resize the image as 420x240
import os
import cv2

for image_name in os.listdir(target_folder):
    img = cv2.imread(os.path.join(target_folder, image_name))
    img = cv2.resize(img, (420, 240))
    cv2.imwrite(os.path.join(target_folder, image_name), img)
print("Finished resizing images")
print(f"Size: {len(os.listdir(target_folder))}")

## Combine the ROAD and RIVER submission

Combine the ROAD and RIVER submission into the `./results/ROAD_RIVER_combined/submission` folder

In [ ]:
source_road_dir = './results/ROAD_pix2pix/test_latest/submission'
source_river_dir = './results/RIVER_pix2pix/test_latest/submission'

target_dir = './results/ROAD_RIVER_combined/submission'

if not os.path.exists(target_dir):
    os.makedirs(target_dir)

for image_name in os.listdir(source_road_dir):
	shutil.copy(os.path.join(source_road_dir, image_name), os.path.join(target_dir, image_name))
for image_name in os.listdir(source_river_dir):
	shutil.copy(os.path.join(source_river_dir, image_name), os.path.join(target_dir, image_name))

In [ ]:
# zip the fake images
shutil.make_archive(target_dir, 'zip', target_dir)